## 국민청원 크롤링하기

In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

import pandas as pd
import re
import time

from datetime import datetime, timedelta
from selenium import webdriver

#### 국민청원 분야별 게시판 주소
- 외교/통일/국방 : 36
- 일자리 : 37
- 인권/성평등 : 49

In [2]:
urls = []
url = input("크롤링할 청원 분야를 입력하세요.\n- 일교/통일/국방 : 36 \n- 일자리 : 37 \n- 인권/성평등 : 49 \n")
for x in range(1, 1000):
    urls.append('https://www1.president.go.kr/petitions/category?c=%s&only=2&page=%d&order=1' % (url, x))

크롤링할 청원 분야를 입력하세요.
- 일교/통일/국방 : 36 
- 일자리 : 37 
- 인권/성평등 : 49 
36


In [3]:
len(urls)

999

In [4]:
urls[0]

'https://www1.president.go.kr/petitions/category?c=36&only=2&page=1&order=1'

### 하나의 페이지 테스트

In [5]:
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)
driver.get(urls[0])
data = BeautifulSoup(driver.page_source, "lxml")

In [6]:
data

<!DOCTYPE html>
<html class=" js csstransitions" lang="ko" style="height: auto;" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta charset="utf-8"/>
<meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0" name="viewport"/>
<meta content="#082e59" name="theme-color"/>
<meta content="나라를 나라답게, 국민과 함께 갑니다." name="description"/>
<meta content="청와대, 대통령비서실, 대통령, 한국, 대한민국, 대한민국 정부, president.go.kr, 靑瓦臺, 19대, 문재인" name="keywords"/>
<meta content="청와대 (CHEONGWADAE)" name="author"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="website" property="og:type"/>
<meta content="분야별 국민청원 및 제안 목록 페이지 1" property="og:title"/>
<meta content="http://www.president.go.kr/" property="og:url"/>
<meta content="분야별 국민청원 및 제안 목록 페이지 1" property="og:site_name"/>
<meta content="나라를 나라답게, 국민과 함께 갑니다." property="og:description"/>
<meta content="http://president.go.kr/css/favicon_400x210.png" prop

In [7]:
data_list = data.find('ul', 'petition_list').find_all('li')

In [8]:
data_list[0]

<li style="padding:20px 0;"><div class="bl_wrap"><div class="bl_no"><span class="sound_only">번호 </span>21301</div><div class="bl_category ccategory cs"><span class="sound_only">분류 </span>외교/통일/국방</div><div class="bl_subject"><a class="cb relpy_w" href="/petitions/408593"><span class="sound_only">제목 </span>9월평양공동 선언 에 대한 국민공청회 를 열어주세요</a></div><div class="bl_name"><span class="sound_only">청원인 </span>naver - ***</div><div class="bl_date light"><span class="sound_only">청원기간 </span>18.10.16 ~ 18.11.15</div><div class="bl_agree cs"><span class="sound_only">참여인원 </span>93명</div></div></li>

In [9]:
data_list[-1]

<li style="padding:20px 0;"><div class="bl_wrap"><div class="bl_no"><span class="sound_only">번호 </span>21292</div><div class="bl_category ccategory cs"><span class="sound_only">분류 </span>외교/통일/국방</div><div class="bl_subject"><a class="cb relpy_w" href="/petitions/408368"><span class="sound_only">제목 </span>조명균은 장관직  사퇴하고 헌법에 보장된 언론 자유를 침해한 죄로 검찰의 수사를 촉구한다ㆍ그리고 굴욕적인 모습을  보인 남북회담을 반대한다</a></div><div class="bl_name"><span class="sound_only">청원인 </span>naver - ***</div><div class="bl_date light"><span class="sound_only">청원기간 </span>18.10.16 ~ 18.11.15</div><div class="bl_agree cs"><span class="sound_only">참여인원 </span>31명</div></div></li>

#### 청원 제목 뽑기

In [10]:
data_list[0].find('a', 'cb relpy_w')

<a class="cb relpy_w" href="/petitions/408593"><span class="sound_only">제목 </span>9월평양공동 선언 에 대한 국민공청회 를 열어주세요</a>

In [11]:
data_list[-1].find('a', 'cb relpy_w')

<a class="cb relpy_w" href="/petitions/408368"><span class="sound_only">제목 </span>조명균은 장관직  사퇴하고 헌법에 보장된 언론 자유를 침해한 죄로 검찰의 수사를 촉구한다ㆍ그리고 굴욕적인 모습을  보인 남북회담을 반대한다</a>

In [12]:
data_list[0].find('a', 'cb relpy_w').get_text()

'제목 9월평양공동 선언 에 대한 국민공청회 를 열어주세요'

In [13]:
data_list[0].find('a', 'cb relpy_w').get_text()[3:]

'9월평양공동 선언 에 대한 국민공청회 를 열어주세요'

#### 청원 날짜 뽑기

In [14]:
data_list[0].find('div', 'bl_date light')

<div class="bl_date light"><span class="sound_only">청원기간 </span>18.10.16 ~ 18.11.15</div>

In [15]:
data_list[-1].find('div', 'bl_date light')

<div class="bl_date light"><span class="sound_only">청원기간 </span>18.10.16 ~ 18.11.15</div>

In [16]:
data_list[0].find('div', 'bl_date light').get_text()

'청원기간 18.10.16 ~ 18.11.15'

In [17]:
data_list[0].find('div', 'bl_date light').get_text()[5:]

'18.10.16 ~ 18.11.15'

#### 청원 참여인원 뽑기

In [18]:
data_list[0].find('div', 'bl_agree cs')

<div class="bl_agree cs"><span class="sound_only">참여인원 </span>93명</div>

In [19]:
data_list[1].find('div', 'bl_agree cs')

<div class="bl_agree cs"><span class="sound_only">참여인원 </span>15명</div>

In [20]:
data_list[0].find('div', 'bl_agree cs').get_text()

'참여인원 93명'

In [21]:
data_list[0].find('div', 'bl_agree cs').get_text()[5:13]

'93명'

### 데이터 수집 및 확인

In [38]:
title = []
date = []
count = []
page = 0

data1 = []

start_flag = False
end_flag = False

end_date = input("시작 날짜를 입력하세요.  ex) 181007 \n")
start_date = input("마지막 날짜를 입력하세요.  ex) 181115 \n")
end_date = end_date[:2] + '.' + end_date[2:4] + '.' + end_date[4:]
start_date = start_date[:2] + '.' + start_date[2:4] + '.' + start_date[4:]

print("=" * 50)
print("[%s ~ %s] 국민청원 데이터를 수집합니다.\n" % (end_date, start_date))

start = datetime.now()

# 가상 드라이버 시작
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)
time.sleep(1)

for url in urls:
    time.sleep(3)
    # url 이동 및 html 소스 수집
    driver.get(url)
    time.sleep(2)
    data = BeautifulSoup(driver.page_source, "lxml")
    time.sleep(2)
    data1.append(data)
    
#     print(data)
    
    # 청원 데이터 찾기
    data_list = data.find('ul', 'petition_list').find_all('li')
#     print(data_list)
    
    # 데이터 수집 체크
    page += 1
#     print(page)
    
    if start_flag:
        print("%d 페이지 수집중입니다.... ing" %page, end='')
    
    for x in range(len(data_list)):
        
        # 기사 끝날 날짜 체크 (end_flag 변경)
        if data_list[x].find('div', 'bl_date light').get_text()[-8:] in end_date:
            temp_date = data_list[x].find('div', 'bl_date light').get_text()[-8:]
            end_flag = True
               

        # 기사 끝날 날짜 체크(반복문 out)    
        if end_flag:
            if not(temp_date == data_list[x].find('div', 'bl_date light').get_text()[-8:]):
                break       
        
        
        if not(start_flag):            
            # 기사 시작 날짜 체크                                                                      
            if not(data_list[x].find('div', 'bl_date light').get_text()[-8:] in start_date):            
                continue
            print("%d 페이지 수집중입니다.... ing" %page, end='')
            start_flag = True        
  
        
        # 청원 제목만 가져오기
        title.append(data_list[x].find('a', 'cb relpy_w').get_text()[3:])
   
        # 청원 날짜만 가져오기
        date.append(data_list[x].find('div', 'bl_date light').get_text()[-8:])
        
        # 청원 참여인원 가져오기
        count.append(data_list[x].find('div', 'bl_agree cs').get_text()[5:])
    
    # 데이터 수집 시작 체크
    if not(start_flag):
        continue
        
    # 데이터수집 체크
    print("     [수집완료]")       
          
    
    # 기사 끝날 날짜 체크(반복문 out)  
    if end_flag:
        if not(temp_date == data_list[x].find('div', 'bl_date light').get_text()[-8:]):
            break 

        
end = datetime.now()
print("=" * 50)
print("데이터 수집을 완료했습니다.")
elapsed = end - start
print('총 계산 시간: ', end='');   print(elapsed)
print('총 수집 개수: %d개' %len(title))    

시작 날짜를 입력하세요.  ex) 181007 
181115
마지막 날짜를 입력하세요.  ex) 181115 
181115
[18.11.15 ~ 18.11.15] 국민청원 데이터를 수집합니다.

1 페이지 수집중입니다.... ing     [수집완료]
2 페이지 수집중입니다.... ing     [수집완료]
3 페이지 수집중입니다.... ing     [수집완료]
4 페이지 수집중입니다.... ing     [수집완료]
5 페이지 수집중입니다.... ing     [수집완료]
데이터 수집을 완료했습니다.
총 계산 시간: 0:00:46.529662
총 수집 개수: 47개


In [22]:
data = BeautifulSoup(driver.page_source, "lxml")

In [35]:
data1[0].find_all("div", "bl_date light")[0].text[-8:]

'18.07.13'

In [40]:
title

['9월평양공동 선언 에 대한 국민공청회 를 열어주세요',
 '남북에철도연결반대한다고역적이라고하는자처벌해주십시요',
 '혈맹인 미국과의 균열조짐 제발 북한에 신경쓰지 마세요.',
 '대북재제',
 '북한편들기  경제파탄 책임 지세요',
 '◈◈ 국제적인 망신__"북한핵" 문제의 당사국인 우리나라가 이상하게 대처하여 동맹국에 까지 항의를 듣다니.......',
 '대통령의 국정 목표가 대북 퍼주기인가?',
 '지금처럼 우리나라 대통령이 부끄러웠던 적이 없었다.',
 '비핵화...왜?  미국인들은 한국을 속국으로 볼까요? 우리는 대한민국입니다',
 '조명균은 장관직  사퇴하고 헌법에 보장된 언론 자유를 침해한 죄로 검찰의 수사를 촉구한다ㆍ그리고 굴욕적인 모습을  보인 남북회담을 반대한다',
 '통일',
 '남북철도 누구 돈으로 하나요?',
 '조명균통일부장관 파면',
 '미세먼지 중국에 대한 항의 및 보상 요구해주세요',
 '장기제대 군인의 대한 공무원 특별 채용을 요청드립니다.',
 '여자남자 성차별 없이 군대가게 해주세요',
 '여적죄(與敵罪)란??',
 '월남참전유공자 전투수당 지급좀 해주십시요',
 '제발 우리 국민의 자존심좀 세워 주세요.',
 '北 김정은 눈치 보며 방어용 무기 개발도 막는 정부 제대로 된 나라인가요.',
 '혼인신고 너무나 쉽다~~ 그러나.......',
 '전상수당 월 2만3천원 주는 대한민국',
 '미국과 북한간 동시 협상 타결',
 '남남갈등을조장하는 탈북민을추방해야합니다.',
 '김정은위원장 방문시 독도방문 제안',
 '북한 외교에 국민세금  쓰지마라',
 '조명균이 문제있어도 해임할 정도는 아니다',
 '대한민국은 전쟁 나면 나를 지켜줄 것입니까?',
 '무슬림난민,불체자,조선족추방,국민시민당(가칭)',
 '본말이 전도된 현 정권의 비핵화 협상!!',
 '군인정신이 투철한 분의 진급에 대한 소',
 '2019년 참전명예수당 동결 이라고 합니다.',
 '통일부 조명균은 당장 사퇴 하라!',
 '역사 교과서에 후쿠자와 유키치를 포

In [18]:
url = "https://www1.president.go.kr/petitions/category?c=36&only=2&page=1&order=1"
html = urlopen(url)
soup = BeautifulSoup(html, "lxml")

In [19]:
soup.find_all("a")

[<a href="#cont_view"><strong>본문으로 바로가기</strong></a>,
 <a href="#sitemap"><strong>하단메뉴 바로가기</strong></a>,
 <a href="/" title="메인페이지로 이동"><img alt="청와대 로고" src="http://www.president.go.kr/cwdimg/main/logo.png" style="width:100%;"/></a>,
 <a href="/calendar">문재인 대통령</a>,
 <a href="/calendar">공개일정</a>,
 <a href="/president-journey">걸어온길</a>,
 <a href="/c/president-speeches">대통령의 말과 글</a>,
 <a href="/c/first-lady-quotations">김정숙의 말과 글</a>,
 <a href="/live-speeches">청와대 뉴스룸</a>,
 <a href="/live-speeches">청와대 온에어</a>,
 <a href="/c/briefings">청와대 브리핑</a>,
 <a href="/c/oversea-journals">해외언론보도</a>,
 <a href="/c/blue-house-stories">청와대 이야기</a>,
 <a href="/TodayShot">오늘의 한 장</a>,
 <a href="/econo_num">숫자로 읽는 우리 경제</a>,
 <a href="/history">청와대 일기</a>,
 <a href="/policies">정책자료</a>,
 <a href="/policies">정책현안</a>,
 <a href="http://www.korea.kr/policy/actuallyList.do?pWise=main&amp;pWiseMain=A25" rel="noopener noreferrer" target="_blank" title="정책브리핑 새창열림">정책브리핑</a>,
 <a href="/government-projects">

In [8]:
len(date)

2507

In [9]:
len(count)

2507

In [10]:
title[0]

'조폭의교사살인, 혐의없음. 재조사할것'

In [11]:
date[0]

'18.11.16 ~ 18.12.16'

In [12]:
count[0]

'0명'

In [13]:
title[-1]

'지하철, 버스 성형 외과 광고 자제 청원'

In [14]:
date[-1]

'18.10.18 ~ 18.11.17'

In [15]:
count[-1]

'0명'

### DataFrame 만들기 & 파일로 저장하기

In [16]:
df_temp = {'Title' : title, 'Date' : date, 'count' : count}
df = pd.DataFrame(df_temp)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2507 entries, 0 to 2506
Data columns (total 3 columns):
Title    2507 non-null object
Date     2507 non-null object
count    2507 non-null object
dtypes: object(3)
memory usage: 58.8+ KB


In [18]:
df.head()

,Title,Date,count
0,"조폭의교사살인, 혐의없음. 재조사할것",18.11.16 ~ 18.12.16,0명
1,"남녀 성 갈등을 유발하는 메갈, 워마드, 일베 등 특정 사이트를 불법으로 지정해주세요",18.11.16 ~ 18.12.16,1명
2,"조폭의교사살인, 혐의없음. 재조사할것",18.11.16 ~ 18.12.16,1명
3,너무 여자들만의 얘기를 들어주니 남자들이 괜한 피해자가 되는 사례가 많이 있네요.,18.11.16 ~ 18.12.16,1명
4,"조폭의교사살인, 혐의없음. 재조사할것",18.11.16 ~ 18.12.16,0명


In [19]:
df.tail()

,Title,Date,count
2502,서울 몰래카메라 강력한 처벌 내려주세요,18.10.18 ~ 18.11.17,895명
2503,언제까지 가해자의 인권을 위한 나라입니까?,18.10.18 ~ 18.11.17,64명
2504,청와대에서 난민 데리고 사세요,18.10.18 ~ 18.11.17,52명
2505,kbs 스페셜 여성 거리에서 외치다 편 방송중단 시켜주세요,18.10.18 ~ 18.11.17,10명
2506,"지하철, 버스 성형 외과 광고 자제 청원",18.10.18 ~ 18.11.17,0명


In [20]:
df.to_csv('TeamMission/president3(%s - %s).csv' % (end_date, start_date), sep=',', encoding='UTF-8')
print('TeamMission/president3(%s - %s).csv 파일을 생성했습니다.' % (end_date, start_date))      

TeamMission/president3(18.01.01 - 18.11.16).csv 파일을 생성했습니다.
